# Boundary conditions

In the previous demo, we showed how using better spatial and temporal discretizations gives more physically realistic simulation results.
The geometry that we used -- a periodic rectangle -- was chosen to be as simple as possible so that we could focus only on certain features of the problem.
For realistic simulations we'll also need to add boundary conditions and in this demo we'll show how.
Correctly implementing and apply boundary conditions is, in our experience, the most common failure mode in scientific computing.

We'll use an ordinary rectangle mesh instead of a periodic one this time.
This is really the only difference between the previous notebook and the current one.

In [ ]:
import firedrake
nx, ny = 24, 24
Lx, Ly = 20., 20.
mesh = firedrake.RectangleMesh(nx, ny, Lx, Ly, diagonal='crossed')

The input data for the simulation will be just like the previous demo.

In [ ]:
from firedrake import inner, max_value, Constant
x = firedrake.SpatialCoordinate(mesh)
lx, ly = 6.0, 5.0
y = Constant((lx, lx))
r = Constant(2.5)

H = Constant(1.0)
δh = Constant(0.1)
h_expr = H + δh * max_value(0, 1 - inner(x - y, x - y) / r**2)

In [ ]:
y = Constant((Lx - lx, Ly - lx))
δb = Constant(1/4)
b = δb * max_value(0, 1 - inner(x - y, x - y) / r**2)

And we'll use the best discretization from the previous demo, a DG(1) element for the thickness and a BDFM(2) element for the momentum.

In [ ]:
degree = 1
Q = firedrake.FunctionSpace(mesh, family='DG', degree=degree)
V = firedrake.FunctionSpace(mesh, family='BDFM', degree=degree + 1)

Z = Q * V
z_0 = firedrake.Function(Z)

h_0, q_0 = z_0.split()
h_0.project(h_expr - b);

In [ ]:
import numpy as np
from plumes.coefficients import gravity
C = np.sqrt(gravity * (float(H) + float(δh)))
δx = mesh.cell_sizes.dat.data_ro[:].min()
timestep = (δx / 4) / C / (2 * degree + 1)

final_time = 4 * Lx / C
num_steps = int(final_time / timestep)
dt = final_time / num_steps

output_time = 1 / 30
output_freq = max(int(output_time / dt), 1)

If we want rigid walls around the boundary of the domain, we don't have to do anything at all!
You can pass inflow and outflow IDs to `shallow_water.make_equation`, but if you don't this routine will assume every other ID is a rigid wall.

In [ ]:
from plumes import models
g = firedrake.Constant(gravity)
equation = models.shallow_water.make_equation(g, b)

The remainder of the simulation and the analysis is just the same as before.

In [ ]:
import tqdm
from plumes import numerics
integrator = numerics.SSPRK34(equation, z_0, dt)

hs = []
qs = []

progress_bar = tqdm.trange(num_steps)
for step in progress_bar:
    if step % output_freq == 0:
        z = integrator.state
        h, q = z.split()
        hmin, hmax = h.dat.data_ro[:].min(), h.dat.data_ro[:].max()
        progress_bar.set_description(f'{hmin:5.3f}, {hmax:5.3f}')
        hs.append(h.copy(deepcopy=True))
        qs.append(q.copy(deepcopy=True))
    
    integrator.step(dt)

Once again, we can see that there's a bit of energy drift but nothing that's cause for too much alarm.

In [ ]:
from firedrake import assemble, dx
energies = np.array([
    0.5 * assemble((inner(q, q) / h + g * (h + b)) * dx)
    for h, q in zip(hs, qs)
])

In [ ]:
import matplotlib.pyplot as plt
fig, axes = plt.subplots()
times = dt * output_freq * np.array(list(range(num_steps // output_freq)))
axes.plot(times, energies)
axes.set_xlabel('time (s)')
axes.set_ylabel('energy');

In the animated results, you can see waves reflecting off of the boundary rather than passing through to the other side, exactly as we expect.

In [ ]:
%%capture
Q0 = firedrake.FunctionSpace(mesh, family='DG', degree=0)
η = firedrake.project(hs[0] + b, Q0)

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

fig, axes = plt.subplots()
axes.set_aspect('equal')
axes.get_xaxis().set_visible(False)
axes.get_yaxis().set_visible(False)
colors = firedrake.tripcolor(
    η, num_sample_points=1, vmin=0.95, vmax=1.05, axes=axes
)
fig.colorbar(colors)

def animate(h):
    η.project(h + b)
    colors.set_array(η.dat.data_ro[:])

interval = 1e3 * output_freq * dt
animation = FuncAnimation(fig, animate, frames=hs, interval=interval)

In [ ]:
from IPython.display import HTML
HTML(animation.to_html5_video())